
# Aircraft Utilization & Scheduling Optimization - SQL Queries

This notebook contains the SQL code used in the Aircraft Utilization and Scheduling Optimization analysis for Flight Schedule Pro.

---

## 🔹 Query 1: Aircraft Utilization Rate

```sql
WITH scheduled_minutes AS (
  SELECT
    aircraft_id,
    DATE(start_time) AS flight_date,
    TIMESTAMP_DIFF(end_time, start_time, MINUTE) AS duration_minutes
  FROM
    `calcium-complex-457419-d6.aircraft_schedules.aircraft_schedules`
  WHERE
    status = 'Completed'
)

SELECT
  aircraft_id,
  COUNT(DISTINCT flight_date) AS active_days,
  SUM(duration_minutes) AS total_scheduled_minutes,
  COUNT(DISTINCT flight_date) * 480 AS total_available_minutes,
  ROUND(SUM(duration_minutes) / (COUNT(DISTINCT flight_date) * 480), 4) AS utilization_rate
FROM
  scheduled_minutes
GROUP BY
  aircraft_id
ORDER BY
  utilization_rate DESC;
```

---

## 🔹 Query 2: Bookings Heatmap (Day of Week × Time Block)

```sql
SELECT
  FORMAT_TIMESTAMP('%A', start_time) AS day_of_week,
  EXTRACT(DAYOFWEEK FROM start_time) AS day_order,

  CASE
    WHEN EXTRACT(HOUR FROM start_time) BETWEEN 6 AND 9 THEN "6–9 AM"
    WHEN EXTRACT(HOUR FROM start_time) BETWEEN 10 AND 13 THEN "10 AM–1 PM"
    WHEN EXTRACT(HOUR FROM start_time) BETWEEN 14 AND 17 THEN "2–5 PM"
    WHEN EXTRACT(HOUR FROM start_time) BETWEEN 18 AND 22 THEN "6–10 PM"
    ELSE "Off Hours"
  END AS time_block,

  CASE
    WHEN EXTRACT(HOUR FROM start_time) BETWEEN 6 AND 9 THEN 1
    WHEN EXTRACT(HOUR FROM start_time) BETWEEN 10 AND 13 THEN 2
    WHEN EXTRACT(HOUR FROM start_time) BETWEEN 14 AND 17 THEN 3
    WHEN EXTRACT(HOUR FROM start_time) BETWEEN 18 AND 22 THEN 4
    ELSE 5
  END AS time_order,

  COUNT(*) AS bookings

FROM
  `calcium-complex-457419-d6.aircraft_schedules.aircraft_schedules`
WHERE
  status = 'Completed'
  AND EXTRACT(HOUR FROM start_time) BETWEEN 6 AND 22
GROUP BY
  day_of_week, day_order, time_block, time_order
ORDER BY
  day_order,
  time_order;
```
